<a href="https://colab.research.google.com/github/gogolucj/class2025Spring/blob/main/naver_cafe_keyword_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __네이버 카페 "신라면세점" 관련 게시글 단어 빈도 분석__

## 1. 라이브러리 설치 및 임포트

In [ ]:
!pip install nltk
!pip install selenium
!pip install konlpy
!apt-get update -qq
!apt-get install -y fonts-nanum

In [ ]:
import numpy as np
import pandas as pd
import re
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import nltk
from nltk import FreqDist
from nltk.text import Text
from konlpy.tag import Okt
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fm.fontManager.addfont(font_path)
plt.rc('font', family='NanumGothic')

## 2. 데이터 크롤링

### (1) 네이버 카페 검색 결과 페이지 크롤링

In [ ]:
# 네이버에서 신라면세점 검색 결과 중 카페 게시글, 최근 1달 이내 게시글
URL = 'https://search.naver.com/search.naver?cafe_where=&date_option=4&prdtype=0&query=%EC%8B%A0%EB%9D%BC%EB%A9%B4%EC%84%B8%EC%A0%90&sm=mtb_opt&ssc=tab.cafe.all&st=rel&stnm=rel&opt_tab=0&nso=so%3Ar%2Cp%3A1m'

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)

driver.get(URL)

all_posts = []

# 스크롤하면 게시글이 계속 추가되는 형태 → 일정 횟수만큼 스크롤 반복
for _ in range(3): # 스크롤 횟수
    # 스크롤 맨 아래로
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 로딩 기다리기
    time.sleep(2)

    # 게시글
    posts = driver.find_elements(By.CLASS_NAME, 'view_wrap')
    for post in posts:
        cafe_name = post.find_element(By.CLASS_NAME, 'user_info').find_element(By.TAG_NAME, 'a').text
        title = post.find_element(By.CLASS_NAME, 'title_area').text
        # summary = post.find_element(By.CLASS_NAME, 'dsc_area').text
        link = post.find_element(By.CLASS_NAME, 'title_area').find_element(By.TAG_NAME, 'a').get_attribute('href')

        post_data = {
            'cafe_name': cafe_name,
            'title': title,
            # 'summary': summary,
            'link': link
        }
        all_posts.append(post_data)
        print(title)
        print('\n')

In [ ]:
len(all_posts)

### (2) 네이버 카페 게시글 크롤링

In [ ]:
for post in all_posts:
    URL = post['link']
    print(URL)
    driver.get(URL)

    time.sleep(2)
    try:
        driver.switch_to.frame('cafe_main')
        content = driver.find_element(By.CLASS_NAME, 'se-main-container').text
        post['content'] = content
    except:
        pass

## 3. 데이터 전처리

In [ ]:
cleaned_contents = []

for post in all_posts:

    try:
        text = post['content']

        pattern = r'^(■).*$' # ■ 로 시작하는 문장 삭제
        text = re.sub(pattern, '', text, flags=re.MULTILINE)
        text = re.sub('구입기 필수 정보', '', text)
        text = re.sub('구매 정보가 없는 글은 회원 술자랑 게시판에 올려주세요', '', text)
        text = re.sub('게시판 이용 주의 사항 - 삭제 금지', '', text)
        text = re.sub(r'[^가-힣a-zA-Z0-9\s\.\,\?\!\$\\\/\:]', '', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\n+', '\n', text).strip()


        # 회사명 통일
        text = text.replace('신라 면세점', '신라면세점')
        text = re.sub(r'\b신라\b', '신라면세점', text)

        cleaned_contents.append(text)
    except:
        pass

In [ ]:
# 품사 분석
joined_contents = ' '.join(cleaned_contents)
okt = Okt()
tokens = okt.pos(joined_contents)
filtered_tokens = [word for word, pos in tokens if pos in ['Noun']]
filtered_tokens = [n for n in filtered_tokens if len(n) > 1] # 2글자 이상 단어

## 4. 데이터 분석

### (1) 빈도 분석

In [ ]:
fdist = FreqDist(filtered_tokens)
remove_words = ['신라', '면세점', '면세']
for word in remove_words:
    if word in fdist:
        del fdist[word]
print(fdist.most_common(10))

In [ ]:
fdist.plot(20)

In [ ]:
wc = WordCloud(font_path = font_path, background_color = 'white', width = 800, height = 400)
wc.generate_from_frequencies(fdist)

plt.figure(figsize = (12, 6))
plt.imshow(wc, interpolation = 'bilinear')
plt.axis('off')
plt.title('신라면세점 게시글 단어 빈도 분석', fontsize = 20)
plt.show()

### (2) 단어 분포 분석

In [ ]:
tokens_list = Text(filtered_tokens)
tokens_list.dispersion_plot(['신라', '롯데', '신세계', '현대', '위스키', '혜택'])